# Part 2: Training and Hosting your Algorithm in Amazon SageMaker

In [1]:
# S3 prefix
prefix = 'DEMO-tensorflow-cifar10'


In [2]:
# The session remembers our connection parameters to SageMaker. We use it to perform all of our SageMaker operations.
import sagemaker as sage

sess = sage.Session()

In [3]:
# Upload the data for training

WORK_DIRECTORY = '/tmp/cifar-10-data'

data_location = sess.upload_data(WORK_DIRECTORY, key_prefix=prefix)

# Training on SageMaker

In [6]:
import boto3
from sagemaker import get_execution_role

role = get_execution_role()

client = boto3.client('sts')
account = client.get_caller_identity()['Account']
print(account)

my_session = boto3.session.Session()
region = my_session.region_name

algorithm_name = 'sagemaker-tf-cifar10-example'

ecr_image = '{}.dkr.ecr.{}.amazonaws.com/{}:latest'.format(account, region, algorithm_name)

print(ecr_image)

731833107751
731833107751.dkr.ecr.us-east-1.amazonaws.com/sagemaker-tf-cifar10-example:latest


In [7]:
from sagemaker.estimator import Estimator

hyperparameters = {'train-steps': 100}

instance_type = 'ml.m5.xlarge'

estimator = Estimator(role=role,
                      train_instance_count=1,
                      train_instance_type=instance_type,
                      image_name=ecr_image,
                      hyperparameters=hyperparameters)

estimator.fit(data_location)

predictor = estimator.deploy(1, instance_type)

2020-02-19 23:10:54 Starting - Starting the training job...
2020-02-19 23:10:56 Starting - Launching requested ML instances......
2020-02-19 23:12:01 Starting - Preparing the instances for training...
2020-02-19 23:12:34 Downloading - Downloading input data......
2020-02-19 23:13:45 Training - Downloading the training image......
2020-02-19 23:14:29 Training - Training image download completed. Training in progress......................................................

KeyboardInterrupt: 

In [ ]:
image = cv2.imread("data/cat.0.jpg", 1)

# resize, as our model is expecting images in 32x32.
image = cv2.resize(image, (32, 32))

data = {'instances': numpy.asarray(image).astype(float).tolist()}

predictor.accept = 'application/json'
predictor.content_type = 'application/json'

predictor.serializer = json_serializer
predictor.deserializer = json_deserializer

predictor.predict(data)